In [1]:
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx
from ItalySetup import ItalySetup
from covidOCP import COVIDVaccinationOCP, COVIDParametersOCP
from main import pick_scenario, build_scenario
import seaborn as sns
import datetime
import pickle
import os

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
when = 'future'
file_prefix = f'week'
outdir = 'helvetios-runs/2021-02-02-107_90/'

generated_dir = 'model_output/scenarios_102_90'

n_int_steps = 6
nnodes = 107  # nodes
ndays_ocp = 90
ndays = 90

setup = ItalySetup(nnodes, ndays, when)
setup_ocp = ItalySetup(nnodes, ndays_ocp, when)
M = setup.nnodes
N = len(setup.model_days) - 1

with open(f'italy-data/parameters_{nnodes}_{when}.pkl', 'rb') as inp:
    p = pickle.load(inp)
    
os.makedirs(f'{generated_dir}', exist_ok=True)

scenarios = {pick_scenario(setup, i)['name']:pick_scenario(setup, i) for i in np.arange(16)}
scenarios.keys()

Loaded Italy Setup with 107 nodes.
Loaded Italy Setup with 107 nodes.


dict_keys(['U-r1-t125000-id0', 'L-r1-t125000-id1', 'U-r1-t250000-id2', 'L-r1-t250000-id3', 'U-r1-t479700-id4', 'L-r1-t479700-id5', 'U-r1-t1000000-id6', 'L-r1-t1000000-id7', 'U-r15-t125000-id8', 'L-r15-t125000-id9', 'U-r15-t250000-id10', 'L-r15-t250000-id11', 'U-r15-t479700-id12', 'L-r15-t479700-id13', 'U-r15-t1000000-id14', 'L-r15-t1000000-id15'])

In [3]:
# Choose a subset of scenarios:
pick = 'r15-'
scenarios = {k:v for (k,v) in scenarios.items() if pick in k}
print(len(scenarios))

8


In [3]:
scenarios_opt = {}
scenarios_baseline = {}
## Re-integrate vacc
for scenario_name, scenario in scenarios.items():
    fname = f"{outdir}{file_prefix}-{scenario_name}-opt-{nnodes}_{ndays_ocp}.csv"  # '-'.join(scenario_name.split('-')[:-1])
    try:
        md = pd.read_csv(fname, index_col= 'date', parse_dates=True)
        print(f'YES {fname}')
    
        # Build scenario
        maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        M = setup.nnodes
        N = setup.ndays - 1
        control_initial = np.zeros((M, N))
        unvac_nd = np.copy(setup.pop_node)
        stockpile = 0
        for k in range(ndays_ocp - 1):
            stockpile += delivery_national[k]
            for nodename in md.place.unique():
                nd = setup.ind2name.index(nodename)
                to_allocate = md[(md['place'] == nodename) & (md['comp'] == 'vacc')].iloc[k]['value']
                to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
                control_initial[nd, k] = to_allocate
                stockpile -= to_allocate
                unvac_nd[nd] -= to_allocate
        p.apply_epicourse(setup, scenario['beta_mult'])
        # END Build scenario

        results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                          setup=setup,
                                                                          parameters=p,
                                                                          controls=control_initial,
                                                                          save_to=f'{generated_dir}/{scenario_name}-opi-{nnodes}_{ndays}',
                                                                          n_rk4_steps=n_int_steps)
        results.set_index('date', drop=True, inplace=True)
        scenarios_opt[scenario_name] = scenario
        print(f'--> DONE {scenario_name}')

        if scenario_name.split('-')[0] not in scenarios_baseline:
            control_initial = np.zeros((M, N))
            # Generate NO vaccination scenarios
            results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-novacc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
            scenarios_baseline[scenario_name.split('-')[0]] = pd.read_csv(f'{generated_dir}/{scenario_name}-novacc-{nnodes}_{ndays}.csv', 
                                                                         index_col= 'date', parse_dates=True)


    except FileNotFoundError:
        print(f'NOT {fname}')

YES helvetios-runs/2021-02-02-107_90/week-U-r15-t125000-id0-opt-107_90.csv


  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/scenarios_102_90/U-r15-t125000-id0-opi-107_90


  2%|▏         | 2/89 [00:00<00:04, 19.20it/s]

--> DONE U-r15-t125000-id0
===> Integrating for model_output/scenarios_102_90/U-r15-t125000-id0-novacc-107_90


...

--> DONE U-r15-t1000000-id12
YES helvetios-runs/2021-02-02-107_90/week-L-r15-t1000000-id13-opt-107_90.csv


  3%|▎         | 3/89 [00:00<00:04, 20.97it/s]

===> Integrating for model_output/scenarios_102_90/L-r15-t1000000-id13-opi-107_90


100%|██████████| 89/89 [00:04<00:00, 21.17it/s]


--> DONE L-r15-t1000000-id13


In [24]:
# REload if restart because of python multiprocessing bug : https://github.com/jupyter/jupyter_client/issues/541
import glob
baseline_fn = glob.glob(f'{generated_dir}/*novacc*.csv')

scenarios_baseline = {}
for fn in baseline_fn:
    scenarios_baseline[fn.split('-')[0][-1]] = pd.read_csv(fn, index_col= 'date', parse_dates=True)
    md = pd.read_csv(fn, index_col= 'date', parse_dates=True)
baseline_fn

['model_output/scenarios_102_90/L-r15-t125000-id1-novacc-107_90.csv',
 'model_output/scenarios_102_90/U-r15-t125000-id0-novacc-107_90.csv']

In [14]:
# Generate alternatives:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())


def generate_all_alternatives(scenario_name, scenario):
    # BY INCIDENCE: 
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['yell'])].groupby('placeID').sum()
    incid.sort_values('value', ascending=False)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp-1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-inc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)

    # BY Suceptibility: 
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['S'])].loc[str(setup.start_date)]
    incid.set_index('placeID', inplace=True)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-sus-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)
    
    
    # BY POPULATION 
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            pop_nd = setup.pop_node[nd]
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * pop_nd / setup.pop_node.sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-pop-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)

    #TODO: Centrality based and R0 based
    return True
    
all_sims = pool.starmap(generate_all_alternatives, [(scenario_name, scenario) for scenario_name, scenario in scenarios.items()])

===> Integrating for model_output/scenarios_102_90/L-r15-t125000-id1-inc-107_90===> Integrating for model_output/scenarios_102_90/U-r15-t1000000-id12-inc-107_90



  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/scenarios_102_90/L-r15-t250000-id5-inc-107_90
===> Integrating for model_output/scenarios_102_90/L-r15-t479700-id9-inc-107_90


  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/scenarios_102_90/U-r15-t479700-id8-inc-107_90===> Integrating for model_output/scenarios_102_90/U-r15-t125000-id0-inc-107_90



...

  0%|          | 0/89 [00:00<?, ?it/s]

===> Integrating for model_output/scenarios_102_90/U-r15-t479700-id8-pop-107_90


  1%|          | 1/89 [00:00<00:11,  7.94it/s]

===> Integrating for model_output/scenarios_102_90/U-r15-t250000-id4-pop-107_90


100%|██████████| 89/89 [00:07<00:00, 11.48it/s]


# Generate the averted cases for all posterior draws

In [15]:
import matlab.engine

eng = matlab.engine.start_matlab()
eng.cd('check-strats/', nargout=0)

eng.workspace['timesV'] = matlab.double(np.arange(738167, 738160+ndays).tolist()) # THIS IS FOR JANUARY 11th

In [16]:
scn_results = pd.DataFrame(columns=['newdoseperweek', 'method', 'infected'])
methods = ['opi','inc', 'pop', 'sus'] # 'opt'

for met in methods:
    for scenario_name, scenario in scenarios.items():
        filename = f'{generated_dir}/{scenario_name}-{met}-{nnodes}_{ndays}.csv'
        if os.path.isfile(filename):
            print(f'Doing {filename}')
            maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
            p.apply_epicourse(setup, scenario['beta_mult'])
            
            md = pd.read_csv(filename, index_col= 'date', parse_dates=True)
            
            dosesV = md[md['comp'] == 'vacc'].pivot(columns='placeID', values='value').to_numpy()

            
            eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
            eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
            eng.run('main_script.m', nargout=0)
            ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
            scn_results = pd.concat([scn_results, pd.DataFrame.from_dict({'newdoseperweek':[int(scenario_name.split('-')[2][1:])]*len(ens_exposed),
                                                                           'method': [met]*len(ens_exposed),
                                                                           'infected':ens_exposed.tolist(),
                                                                           'post_sample':np.arange(len(ens_exposed)),
                                                                           'doses': [dosesV.sum()]*len(ens_exposed),
                                                                           'scenario-beta': [scenario_name.split('-')[0]]*len(ens_exposed),
                                                                           'scenario-rate': [scenario_name.split('-')[1]]*len(ens_exposed),
                                                                           'scenario-tot': [scenario_name.split('-')[2]]*len(ens_exposed),
                                                                           'scenario': [scenario_name]*len(ens_exposed)
                                                                         })])
        else:
            print(f'not found {scenario_name}')

Doing model_output/scenarios_102_90/U-r15-t125000-id0-opi-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t125000-id1-opi-107_90.csv
Doing model_output/scenarios_102_90/U-r15-t250000-id4-opi-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t250000-id5-opi-107_90.csv
Doing model_output/scenarios_102_90/U-r15-t479700-id8-opi-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t479700-id9-opi-107_90.csv
Doing model_output/scenarios_102_90/U-r15-t1000000-id12-opi-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t1000000-id13-opi-107_90.csv
Doing model_output/scenarios_102_90/U-r15-t125000-id0-inc-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t125000-id1-inc-107_90.csv
Doing model_output/scenarios_102_90/U-r15-t250000-id4-inc-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t250000-id5-inc-107_90.csv
Doing model_output/scenarios_102_90/U-r15-t479700-id8-inc-107_90.csv
Doing model_output/scenarios_102_90/L-r15-t479700-id9-inc-107_90.csv
Doing model_output/scenarios_1

In [17]:
scenarios_baseline = {}
for scenario_name, scenario in scenarios.items():
    if scenario_name.split('-')[0] not in scenarios_baseline:
        print(scenario_name)

        maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        
        p.apply_epicourse(setup, scenario['beta_mult'])


        dosesV = np.zeros_like(dosesV)


        eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
        eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
        eng.run('main_script.m', nargout=0)
        ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
        scenarios_baseline[scenario_name.split('-')[0]] = ens_exposed
        

U-r15-t125000-id0
L-r15-t125000-id1


In [23]:
scn_results

,newdoseperweek,method,infected,post_sample,doses,scenario-beta,scenario-rate,scenario-tot,scenario,bl,averted,avertedpervacc
0,125000,opi,1.149170e+08,0.0,1.629663e+06,U,r15,t125000,U-r15-t125000-id0,1.168417e+08,1.924678e+06,1.181029
1,125000,opi,1.585646e+08,1.0,1.629663e+06,U,r15,t125000,U-r15-t125000-id0,1.602782e+08,1.713631e+06,1.051525
2,125000,opi,5.315905e+07,2.0,1.629663e+06,U,r15,t125000,U-r15-t125000-id0,5.366247e+07,5.034211e+05,0.308911
3,125000,opi,1.597302e+08,3.0,1.629663e+06,U,r15,t125000,U-r15-t125000-id0,1.630720e+08,3.341819e+06,2.050620
4,125000,opi,9.898532e+07,4.0,1.629663e+06,U,r15,t125000,U-r15-t125000-id0,9.935567e+07,3.703520e+05,0.227257
...,...,...,...,...,...,...,...,...,...,...,...,...
97,1000000,sus,3.632644e+07,97.0,1.300658e+07,L,r15,t1000000,L-r15-t1000000-id13,3.705820e+07,7.317600e+05,0.056261
98,1000000,sus,3.258646e+07,98.0,1.300658e+07,L,r15,t1000000,L-r15-t1000000-id13,3.304409e+07,4.576227e+05,0.035184
99,1000000,sus,3.652961e+07,99.0,1.300658e+07,L,r15,t1000000,L-r15-t1000000-id13,3.710113e+07,5.715172e+05,0.043941
100,1000000,sus,2.963616e+07,100.0,1.300658e+07,L,r15,t1000000,L-r15-t1000000-id13,3.000549e+07,3.693269e+05,0.028395


In [19]:
nbtileU = int(len(scn_results.loc[scn_results['scenario-beta'] == 'U', 'scenario-tot'])/len(scenarios_baseline['U']))
nbtileL = int(len(scn_results.loc[scn_results['scenario-beta'] == 'L', 'scenario-tot'])/len(scenarios_baseline['L']))

In [20]:
scn_results['bl'] = 0

scn_results.loc[scn_results['scenario-beta'] == 'U', 'bl'] = np.tile(scenarios_baseline['U'],nbtileU)
scn_results.loc[scn_results['scenario-beta'] == 'L', 'bl'] = np.tile(scenarios_baseline['L'],nbtileL)

In [21]:
scn_results['averted'] = scn_results['bl'] - scn_results['infected']
scn_results['avertedpervacc'] = scn_results['averted']/scn_results['doses']

In [22]:
scn_results.to_csv(f'{generated_dir}/all_summary.csv')

In [4]:
scenarios_specs = {
'vaccpermonthM': [1.5, 15, 150],  # ax.set_ylim(0.05, 0.4)
# 'vacctotalM': [2, 5, 10, 15, 20],
'newdoseperweek': [125000, 250000, 479700, 1e6],
'epicourse': ['U', 'L']  # 'U'
}

import itertools
# Compute all permutatios
keys, values = zip(*scenarios_specs.items())
permuted_specs = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [15]:
specs = pd.DataFrame.from_dict(permuted_specs)
specs = specs[((specs['vaccpermonthM'] == 15.0) | (specs['newdoseperweek'] == 479700.0))].reset_index(drop = True)

,vaccpermonthM,newdoseperweek,epicourse
0,1.5,479700.0,U
1,1.5,479700.0,L
2,15.0,125000.0,U
3,15.0,125000.0,L
4,15.0,250000.0,U
5,15.0,250000.0,L
6,15.0,479700.0,U
7,15.0,479700.0,L
8,15.0,1000000.0,U
9,15.0,1000000.0,L


In [19]:
specs.to_csv('scenarios_specs_2021-02-12.csv')